# Experiment 1  (Adding new ROI Head)

Using `bdd100k_24.pth` as S and performing training on IDD

In [1]:
from imports import *
from datasets.idd import *
from datasets.bdd import *
from collections import OrderedDict
from cfg import *

#from fastprogress import master_bar, progress_bar

In [2]:
batch_size=8

In [3]:
with open("datalists/idd_images_path_list.txt", "rb") as fp:
    non_hq_img_paths = pickle.load(fp)
with open("datalists/idd_anno_path_list.txt", "rb") as fp:
    non_hq_anno_paths = pickle.load(fp)

with open("datalists/idd_hq_images_path_list.txt", "rb") as fp:
    hq_img_paths = pickle.load(fp)
with open("datalists/idd_hq_anno_path_list.txt", "rb") as fp:
    hq_anno_paths = pickle.load(fp)
    
trgt_images =   non_hq_img_paths #hq_img_paths
trgt_annos = non_hq_anno_paths #hq_anno_paths + hq_anno_paths 
trgt_dataset = IDD(trgt_images,trgt_annos,get_transform(train=True))
trgt_dl =  torch.utils.data.DataLoader(trgt_dataset, batch_size=batch_size, shuffle=True, num_workers=4,collate_fn=utils.collate_fn)

In [4]:
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True).cpu()
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes).cpu() # replace the pre-trained head with a new one
    return model.cpu()

In [5]:
# ckpt = torch.load('saved_models/ulm_det_ft0.pth')
# model = get_model(15)
# model.load_state_dict(ckpt['model'])

## 1.1 Using the baseline model and changing the ROI Head

In [6]:
model = get_model(12)
ckpt = torch.load('saved_models/bdd100k_24.pth')
model.load_state_dict(ckpt['model'])

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model.roi_heads.box_predictor = FastRCNNPredictor(1024, 15)
model.cuda();

<All keys matched successfully>

In [7]:
# model.roi_heads.load_state_dict(model_bdd.roi_heads.state_dict())

In [9]:
for n,p in model.named_parameters():
    p.requires_grad=False             #Number of params in RPN = 593935
    
for n,p in model.roi_heads.named_parameters():
    p.requires_grad=True             #Number of params in RPN = 593935

In [10]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=1e-3,
                            momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer,base_lr=1e-3,max_lr=6e-3)

In [12]:
# evaluate(model,val_dl,device=torch.device('cuda'))

In [16]:
with open("datalists/idd_val_images_path_list.txt", "rb") as fp:
    val_img_paths = pickle.load(fp)
    
with open("datalists/idd_val_anno_path_list.txt", "rb") as fp:
    val_anno_paths = pickle.load(fp)
#val_img_paths = val_img_paths[:10]
#val_anno_paths = val_anno_paths[:10]
val_dataset = IDD_Test(val_img_paths,val_anno_paths)
val_dl =  torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4,collate_fn=utils.collate_fn)

In [14]:
device=torch.device('cuda')

In [15]:
for epoch in tqdm(range(num_epochs)):
    train_one_epoch(model, optimizer, trgt_dl, device, epoch, print_freq=200)
    
    lr_scheduler.step()
    
    if epoch==5 or epoch==10 or epoch==15 or epoch == 20 or epoch==24:
        save_name = 'saved_models/s_bdd_t_idd_task1_' + str(epoch)+'.pth'
        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }, save_name)
        print("Saved model", save_name)
        print("Evaluation in progress")
        evaluate(model,val_dl,device=torch.device('cuda'))

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch: [0]  [   0/3384]  eta: 4:20:05  lr: 0.000002  loss: 3.0987 (3.0987)  loss_classifier: 2.6793 (2.6793)  loss_box_reg: 0.2806 (0.2806)  loss_objectness: 0.0503 (0.0503)  loss_rpn_box_reg: 0.0885 (0.0885)  time: 4.6116  data: 1.6654  max mem: 8738
Epoch: [0]  [ 200/3384]  eta: 2:09:28  lr: 0.000202  loss: 0.8891 (1.4638)  loss_classifier: 0.4102 (1.0307)  loss_box_reg: 0.2505 (0.2563)  loss_objectness: 0.0503 (0.0817)  loss_rpn_box_reg: 0.0944 (0.0952)  time: 2.4173  data: 0.0418  max mem: 8873
Epoch: [0]  [ 400/3384]  eta: 2:01:11  lr: 0.000402  loss: 0.6884 (1.1298)  loss_classifier: 0.3163 (0.7033)  loss_box_reg: 0.2295 (0.2475)  loss_objectness: 0.0516 (0.0806)  loss_rpn_box_reg: 0.0911 (0.0983)  time: 2.4347  data: 0.0407  max mem: 8873
Epoch: [0]  [ 600/3384]  eta: 1:53:02  lr: 0.000601  loss: 0.6451 (0.9855)  loss_classifier: 0.2926 (0.5734)  loss_box_reg: 0.2043 (0.2370)  loss_objectness: 0.0514 (0.0777)  loss_rpn_box_reg: 0.0834 (0.0974)  time: 2.4264  data: 0.0410  max me

  4%|▍         | 1/25 [2:17:20<54:56:09, 8240.40s/it]

Epoch: [0]  [3383/3384]  eta: 0:00:02  lr: 0.001000  loss: 0.4909 (0.6167)  loss_classifier: 0.2251 (0.2918)  loss_box_reg: 0.1215 (0.1489)  loss_objectness: 0.0521 (0.0774)  loss_rpn_box_reg: 0.0882 (0.0984)  time: 2.4425  data: 0.0399  max mem: 8873
Epoch: [0] Total time: 2:17:20 (2.4351 s / it)
Epoch: [1]  [   0/3384]  eta: 3:40:03  lr: 0.001003  loss: 0.4161 (0.4161)  loss_classifier: 0.2009 (0.2009)  loss_box_reg: 0.0721 (0.0721)  loss_objectness: 0.0761 (0.0761)  loss_rpn_box_reg: 0.0670 (0.0670)  time: 3.9018  data: 1.4479  max mem: 8873
Epoch: [1]  [ 200/3384]  eta: 2:09:37  lr: 0.001003  loss: 0.4561 (0.5064)  loss_classifier: 0.1917 (0.2142)  loss_box_reg: 0.0978 (0.1127)  loss_objectness: 0.0497 (0.0766)  loss_rpn_box_reg: 0.0925 (0.1029)  time: 2.4364  data: 0.0428  max mem: 8873
Epoch: [1]  [ 400/3384]  eta: 2:01:20  lr: 0.001003  loss: 0.4363 (0.4941)  loss_classifier: 0.1780 (0.2108)  loss_box_reg: 0.0938 (0.1108)  loss_objectness: 0.0459 (0.0736)  loss_rpn_box_reg: 0.10

  8%|▊         | 2/25 [4:34:38<52:38:33, 8239.71s/it]

...


100%|██████████| 10/10 [00:00<00:00, 31.03it/s]

creating index...
index created!
Test:  [0/2]  eta: 0:00:02  model_time: 0.4576 (0.4576)  evaluator_time: 0.0030 (0.0030)  time: 1.0782  data: 0.5876  max mem: 8873


 24%|██▍       | 6/25 [13:43:38<43:28:02, 8235.93s/it]

Test:  [1/2]  eta: 0:00:00  model_time: 0.3037 (0.3807)  evaluator_time: 0.0013 (0.0022)  time: 0.7001  data: 0.2954  max mem: 8873
Test: Total time: 0:00:01 (0.7197 s / it)
Averaged stats: model_time: 0.3037 (0.3807)  evaluator_time: 0.0013 (0.0022)
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.925
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.925
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.950
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.950
 Average Recall     (AR) @[ IoU=0.50:0.

KeyboardInterrupt: 

In [17]:
evaluate(model,val_dl,device=torch.device('cuda'))


 11%|█▏        | 1156/10225 [00:34<06:59, 21.63it/s]


 20%|█▉        | 2032/10225 [01:13<06:06, 22.34it/s]


 28%|██▊       | 2874/10225 [01:53<05:06, 24.02it/s]


 35%|███▍      | 3562/10225 [02:37<08:45, 12.68it/s]


 41%|████      | 4171/10225 [03:22<06:34, 15.35it/s]


...

 83%|████████▎ | 8464/10225 [07:36<01:20, 21.74it/s]


 91%|█████████ | 9325/10225 [08:17<00:43, 20.64it/s]


 99%|█████████▉| 10150/10225 [08:56<00:03, 22.06it/s]


100%|██████████| 10225/10225 [08:59<00:00, 23.05it/s]


creating index...
index created!
Test:  [   0/1279]  eta: 0:32:20  model_time: 0.3901 (0.3901)  evaluator_time: 0.0197 (0.0197)  time: 1.5172  data: 1.0775  max mem: 8876
Test:  [ 100/1279]  eta: 0:11:14  model_time: 0.3425 (0.3392)  evaluator_time: 0.0081 (0.0141)  time: 0.5534  data: 0.1805  max mem: 8876
Test:  [ 200/1279]  eta: 0:10:16  model_time: 0.3439 (0.3408)  evaluator_time: 0.0079 (0.0132)  time: 0.5656  data: 0.1850  max mem: 8876
Test:  [ 300/1279]  eta: 0:09:18  model_time: 0.3465 (0.3415)  evaluator_time: 0.0077 (0.0128)  time: 0.5645  data: 0.1772  max mem: 8876
Test:  [ 400/1279]  eta: 0:08:20  model_time: 0.3468 (0.3415)  evaluator_time: 0.0096 (0.0129)  time: 0.5673  data: 0.1803  max mem: 8876
Test:  [ 500/1279]  eta: 0:07:22  model_time: 0.3421 (0.3413)  evaluator_time: 0.0090 (0.0129)  time: 0.5624  data: 0.1740  max mem: 8876
Test:  [ 600/1279]  eta: 0:06:25  model_time: 0.3446 (0.3415)  evaluator_time: 0.0074 (0.0128)  time: 0.5679  data: 0.1827  max mem: 8876
T

## 1.2 Checking on S (with ROI Head 1)

In [ ]:
model.roi_heads.box_predictor = FastRCNNPredictor(1024, 12)

model_bdd = get_model(12)
checkpoint = torch.load('saved_models/'+'bdd100k_24.pth')
model_bdd.load_state_dict(checkpoint['model'])

model.roi_heads.load_state_dict(model_bdd.roi_heads.state_dict())

model.cuda();

In [ ]:
root_img_path = os.path.join(bdd_path,'images','100k')
root_anno_path = os.path.join(bdd_path,'labels')

val_img_path = root_img_path+'/val/'
val_anno_json_path = root_anno_path+'/bdd100k_labels_images_val.json'

with open("datalists/bdd100k_val_images_path.txt", "rb") as fp:
    bdd_img_path_list = pickle.load(fp)
#bdd_img_path_list = bdd_img_path_list[:10]
val_dataset = BDD(bdd_img_path_list,val_anno_json_path)
val_dl =  torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=0,collate_fn=utils.collate_fn,pin_memory=True)

In [ ]:
for n,p in model.named_parameters():
    p.requires_grad=False             #Number of params in RPN = 593935
    
for n,p in model.roi_heads.named_parameters():
    p.requires_grad=True             #Number of params in RPN = 593935
    
evaluate(model,val_dl,device=torch.device('cuda'))

# Experiment 2 
### 2.1 Discriminative FT+Grad freezing

In [1]:
from imports import *
from datasets.idd import *
from datasets.bdd import *
from collections import OrderedDict
from cfg import *

In [2]:
batch_size=16

In [3]:
with open("datalists/idd_images_path_list.txt", "rb") as fp:
    non_hq_img_paths = pickle.load(fp)
with open("datalists/idd_anno_path_list.txt", "rb") as fp:
    non_hq_anno_paths = pickle.load(fp)

with open("datalists/idd_hq_images_path_list.txt", "rb") as fp:
    hq_img_paths = pickle.load(fp)
with open("datalists/idd_hq_anno_path_list.txt", "rb") as fp:
    hq_anno_paths = pickle.load(fp)
    
trgt_images =   non_hq_img_paths #hq_img_paths
trgt_annos = non_hq_anno_paths #hq_anno_paths + hq_anno_paths 
trgt_dataset = IDD(trgt_images,trgt_annos,get_transform(train=True))
trgt_dl =  torch.utils.data.DataLoader(trgt_dataset, batch_size=batch_size, shuffle=True, num_workers=4,collate_fn=utils.collate_fn)

In [4]:
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True).cpu()
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes).cpu() # replace the pre-trained head with a new one
    return model.cpu()

In [5]:
model = get_model(15)
ckpt = torch.load('saved_models/task_2_1/s_bdd_t_idd_task_new_2_1_epoch_+4.pth')
model.load_state_dict(ckpt['model'])

<All keys matched successfully>

In [6]:
for n,p in model.named_parameters():
    p.requires_grad=False             #Number of params in RPN = 593935
    
for n,p in model.rpn.named_parameters():
    p.requires_grad=True   
    
for n,p in model.roi_heads.named_parameters():
    p.requires_grad=True             #Number of params in RPN = 593935

In [7]:
device=torch.device('cuda')
model.to(device);

In [6]:
optimizer = torch.optim.SGD([
                {'params': model.backbone.parameters(),'lr':1e-5},
                {'params': model.rpn.parameters(), 'lr':6e-4},
                {'params': model.roi_heads.parameters(), 'lr': 2e-3}
            ])

In [7]:
optimizer.load_state_dict(ckpt['optimizer'])

In [10]:
lr_scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer,base_lr=1e-4,max_lr=6e-3)

In [11]:
with open("datalists/idd_val_images_path_list.txt", "rb") as fp:
    val_img_paths = pickle.load(fp)
    
with open("datalists/idd_val_anno_path_list.txt", "rb") as fp:
    val_anno_paths = pickle.load(fp)
#val_img_paths = val_img_paths[:10]
#val_anno_paths = val_anno_paths[:10]
val_dataset = IDD(val_img_paths,val_anno_paths)
val_dl =  torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4,collate_fn=utils.collate_fn)

In [12]:
for epoch in tqdm(range(12,21)):
    train_one_epoch(model, optimizer, trgt_dl, device, epoch, print_freq=200)
    
    lr_scheduler.step()
    
    save_name = 'saved_models/task_2_1/s_bdd_t_idd_task_new_2_1_epoch_+' + str(epoch)+'.pth'
    torch.save({
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        }, save_name)
    print("Saved model", save_name)
    
#     if epoch==5 or epoch==10 or epoch==15 or epoch==20:
#         print("Evaluation in progress")
#         evaluate(model,val_dl,device=torch.device('cuda'))

  0%|          | 0/17 [00:00<?, ?it/s]

Epoch: [4]  [   0/1692]  eta: 2:55:16  loss_rpn_box_reg: 0.0633 (0.0633)  loss_classifier: 0.1700 (0.1700)  lr: 0.000100  loss_objectness: 0.0425 (0.0425)  loss: 0.3632 (0.3632)  loss_box_reg: 0.0874 (0.0874)  time: 6.2154  data: 4.1949  max mem: 12225
Epoch: [4]  [ 200/1692]  eta: 0:38:33  loss_rpn_box_reg: 0.0847 (0.0913)  loss_classifier: 0.1820 (0.1791)  lr: 0.000100  loss_objectness: 0.0524 (0.0555)  loss: 0.4302 (0.4234)  loss_box_reg: 0.0944 (0.0975)  time: 1.5297  data: 0.0743  max mem: 12319
Epoch: [4]  [ 400/1692]  eta: 0:33:06  loss_rpn_box_reg: 0.0682 (0.0889)  loss_classifier: 0.1590 (0.1784)  lr: 0.000100  loss_objectness: 0.0465 (0.0552)  loss: 0.3904 (0.4191)  loss_box_reg: 0.0925 (0.0966)  time: 1.5197  data: 0.0758  max mem: 12319
Epoch: [4]  [ 600/1692]  eta: 0:27:54  loss_rpn_box_reg: 0.0804 (0.0896)  loss_classifier: 0.1627 (0.1780)  lr: 0.000100  loss_objectness: 0.0495 (0.0545)  loss: 0.3885 (0.4185)  loss_box_reg: 0.0856 (0.0963)  time: 1.5328  data: 0.0766  max

  6%|▌         | 1/17 [43:07<11:29:59, 2587.47s/it]

Saved model saved_models/task_2_1/s_bdd_t_idd_task_new_2_1_epoch_+4.pth
Epoch: [5]  [   0/1692]  eta: 2:41:45  loss_rpn_box_reg: 0.0998 (0.0998)  loss_classifier: 0.1954 (0.1954)  lr: 0.000103  loss_objectness: 0.0505 (0.0505)  loss: 0.4579 (0.4579)  loss_box_reg: 0.1123 (0.1123)  time: 5.7362  data: 4.2408  max mem: 12319
Epoch: [5]  [ 200/1692]  eta: 0:38:21  loss_rpn_box_reg: 0.0739 (0.0844)  loss_classifier: 0.1824 (0.1749)  lr: 0.000103  loss_objectness: 0.0452 (0.0534)  loss: 0.4122 (0.4077)  loss_box_reg: 0.1000 (0.0950)  time: 1.5223  data: 0.0766  max mem: 12319
Epoch: [5]  [ 400/1692]  eta: 0:33:02  loss_rpn_box_reg: 0.0918 (0.0871)  loss_classifier: 0.1657 (0.1768)  lr: 0.000103  loss_objectness: 0.0429 (0.0540)  loss: 0.4147 (0.4135)  loss_box_reg: 0.0914 (0.0956)  time: 1.5195  data: 0.0768  max mem: 12319
Epoch: [5]  [ 600/1692]  eta: 0:27:53  loss_rpn_box_reg: 0.0955 (0.0885)  loss_classifier: 0.1768 (0.1777)  lr: 0.000103  loss_objectness: 0.0505 (0.0554)  loss: 0.4220 

 12%|█▏        | 2/17 [1:26:13<10:46:47, 2587.14s/it]

...

 41%|████      | 7/17 [5:01:49<7:11:17, 2587.76s/it]

Saved model saved_models/task_2_1/s_bdd_t_idd_task_new_2_1_epoch_+10.pth
Epoch: [11]  [   0/1692]  eta: 2:37:54  loss_rpn_box_reg: 0.0621 (0.0621)  loss_classifier: 0.1582 (0.1582)  lr: 0.000121  loss_objectness: 0.0422 (0.0422)  loss: 0.3746 (0.3746)  loss_box_reg: 0.1121 (0.1121)  time: 5.5999  data: 4.1021  max mem: 12319
Epoch: [11]  [ 200/1692]  eta: 0:38:27  loss_rpn_box_reg: 0.0805 (0.0862)  loss_classifier: 0.1876 (0.1788)  lr: 0.000121  loss_objectness: 0.0503 (0.0528)  loss: 0.4206 (0.4164)  loss_box_reg: 0.0960 (0.0986)  time: 1.5236  data: 0.0749  max mem: 12319
Epoch: [11]  [ 400/1692]  eta: 0:33:03  loss_rpn_box_reg: 0.0802 (0.0867)  loss_classifier: 0.1821 (0.1771)  lr: 0.000121  loss_objectness: 0.0498 (0.0515)  loss: 0.4133 (0.4128)  loss_box_reg: 0.0967 (0.0974)  time: 1.5121  data: 0.0735  max mem: 12319
Epoch: [11]  [ 600/1692]  eta: 0:27:53  loss_rpn_box_reg: 0.0969 (0.0862)  loss_classifier: 0.1810 (0.1773)  lr: 0.000121  loss_objectness: 0.0513 (0.0518)  loss: 0.

 47%|████▋     | 8/17 [5:44:56<6:28:06, 2587.40s/it]

Saved model saved_models/task_2_1/s_bdd_t_idd_task_new_2_1_epoch_+11.pth
Epoch: [12]  [   0/1692]  eta: 2:17:03  loss_rpn_box_reg: 0.0919 (0.0919)  loss_classifier: 0.1723 (0.1723)  lr: 0.000124  loss_objectness: 0.0337 (0.0337)  loss: 0.3992 (0.3992)  loss_box_reg: 0.1013 (0.1013)  time: 4.8604  data: 3.3635  max mem: 12319
Epoch: [12]  [ 200/1692]  eta: 0:38:26  loss_rpn_box_reg: 0.0841 (0.0873)  loss_classifier: 0.1587 (0.1755)  lr: 0.000124  loss_objectness: 0.0451 (0.0503)  loss: 0.3917 (0.4092)  loss_box_reg: 0.0846 (0.0961)  time: 1.5235  data: 0.0746  max mem: 12319
Epoch: [12]  [ 400/1692]  eta: 0:33:03  loss_rpn_box_reg: 0.0743 (0.0883)  loss_classifier: 0.1627 (0.1788)  lr: 0.000124  loss_objectness: 0.0478 (0.0518)  loss: 0.3927 (0.4168)  loss_box_reg: 0.0944 (0.0978)  time: 1.5328  data: 0.0880  max mem: 12319
Epoch: [12]  [ 600/1692]  eta: 0:27:53  loss_rpn_box_reg: 0.0830 (0.0882)  loss_classifier: 0.1850 (0.1776)  lr: 0.000124  loss_objectness: 0.0451 (0.0516)  loss: 0.

KeyboardInterrupt: 

## Running Evaluation

In [1]:
from imports import *
from datasets.idd import *
from datasets.bdd import *
from collections import OrderedDict
from cfg import *
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

batch_size=16
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True).cpu()
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes).cpu() # replace the pre-trained head with a new one
    return model.cpu()

In [2]:
with open("datalists/idd_val_images_path_list.txt", "rb") as fp:
    val_img_paths = pickle.load(fp)
    
with open("datalists/idd_val_anno_path_list.txt", "rb") as fp:
    val_anno_paths = pickle.load(fp)
#val_img_paths = val_img_paths[:10]
#val_anno_paths = val_anno_paths[:10]
val_dataset_idd = IDD(val_img_paths,val_anno_paths)
val_dl_idd =  torch.utils.data.DataLoader(val_dataset_idd, batch_size=batch_size, shuffle=True, num_workers=4,collate_fn=utils.collate_fn)

In [3]:
root_img_path = os.path.join(bdd_path,'images','100k')
root_anno_path = os.path.join(bdd_path,'labels')

val_img_path = root_img_path+'/val/'
val_anno_json_path = root_anno_path+'/bdd100k_labels_images_val.json'

with open("datalists/bdd100k_val_images_path.txt", "rb") as fp:
    bdd_img_path_list = pickle.load(fp)
#bdd_img_path_list = bdd_img_path_list[:10]
val_dataset_bdd = BDD(bdd_img_path_list,val_anno_json_path)
val_dl_bdd =  torch.utils.data.DataLoader(val_dataset_bdd, batch_size=batch_size, shuffle=True, num_workers=0,collate_fn=utils.collate_fn,pin_memory=True)

100%|██████████| 10000/10000 [00:00<00:00, 25549.10it/s]


In [4]:
coco_idd = get_coco_api_from_dataset(val_dl_idd.dataset)
coco_bdd = get_coco_api_from_dataset(val_dl_bdd.dataset)

  0%|          | 0/10000 [00:00<?, ?it/s]

creating index...
index created!


100%|██████████| 10000/10000 [04:23<00:00, 38.02it/s]


creating index...
index created!


In [5]:
@torch.no_grad()
def evaluate_(model,coco_dset,data_loader, device):
    iou_types = ["bbox"]
    coco = coco_dset
    n_threads = torch.get_num_threads()
    # FIXME remove this and make paste_masks_in_image run on the GPU
    torch.set_num_threads(1)
    cpu_device = torch.device("cpu")
    model.eval()
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Test:'
    model.to(device)
    iou_types = _get_iou_types(model)
    coco_evaluator = CocoEvaluator(coco, iou_types)
    to_tensor = torchvision.transforms.ToTensor()
    for image, targets in metric_logger.log_every(data_loader, 100, header):
        
        image = list(to_tensor(img).to(device) for img in image)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        torch.cuda.synchronize()
        model_time = time.time()
        
        outputs = model(image)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        evaluator_time = time.time()
        coco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    coco_evaluator.synchronize_between_processes()

    # accumulate predictions from all images
    coco_evaluator.accumulate()
    coco_evaluator.summarize()
    torch.set_num_threads(n_threads)
    return coco_evaluator    

def _get_iou_types(model):
    model_without_ddp = model
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_without_ddp = model.module
    iou_types = ["bbox"]
    return iou_types

In [6]:
 device=torch.device('cuda')

In [7]:
trained_models = [  'task_2_1/s_bdd_t_idd_task_new_2_1_epoch_0.pth',
                  'task_2_1/s_bdd_t_idd_task_new_2_1_epoch_1.pth',
                  'task_2_1/s_bdd_t_idd_task_new_2_1_epoch_2.pth',
                  'task_2_1/s_bdd_t_idd_task_new_2_1_epoch_3.pth',
                  'task_2_1/s_bdd_t_idd_task_new_2_1_epoch_4.pth',
                  'task_2_1/s_bdd_t_idd_task_new_2_1_epoch_5.pth',
                  'task_2_1/s_bdd_t_idd_task_new_2_1_epoch_6.pth',
                  'task_2_1/s_bdd_t_idd_task_new_2_1_epoch_7.pth',
                  'task_2_1/s_bdd_t_idd_task_new_2_1_epoch_8.pth',
                  'task_2_1/s_bdd_t_idd_task_new_2_1_epoch_9.pth',
                  'task_2_1/s_bdd_t_idd_task_new_2_1_epoch_10.pth',
                  
                ]

In [ ]:
for idx in tqdm(range(5,len(trained_models))):
    model = get_model(15)
    ckpt = torch.load('saved_models/'+trained_models[idx])
    model.load_state_dict(ckpt['model'])
    
    model.to(device);
    
    print("##########  Evaluation of IDD  ", "###   IDX  ",idx)
    
    evaluate_(model,coco_idd,val_dl_idd,device=torch.device('cuda'))
    
    model.roi_heads.box_predictor = FastRCNNPredictor(1024, 12)

    model_bdd = get_model(12)
    checkpoint = torch.load('saved_models/'+'bdd100k_24.pth')
    model_bdd.load_state_dict(checkpoint['model'])

    model.roi_heads.load_state_dict(model_bdd.roi_heads.state_dict())

    model.cuda();
    
    for n,p in model.named_parameters():
        p.requires_grad=False             #Number of params in RPN = 593935
    
    for n,p in model.rpn.named_parameters():
        p.requires_grad=True  
    
    for n,p in model.roi_heads.named_parameters():
        p.requires_grad=True             #Number of params in RPN = 593935
    
    print("##########  Evaluation of BDD  ", "###   IDX  ",idx)
    evaluate_(model,coco_bdd,val_dl_bdd,device=torch.device('cuda'))
    
    del model,model_bdd

  0%|          | 0/6 [00:00<?, ?it/s]

##########  Evaluation of IDD   ###   IDX   5
Test:  [  0/640]  eta: 1:17:55  model_time: 3.2593 (3.2593)  evaluator_time: 0.1353 (0.1353)  time: 7.3051  data: 3.4681  max mem: 11740
Test:  [100/640]  eta: 0:17:21  model_time: 1.0309 (1.0478)  evaluator_time: 0.1069 (0.1164)  time: 1.8787  data: 0.3789  max mem: 12207
Test:  [200/640]  eta: 0:13:57  model_time: 1.0318 (1.0369)  evaluator_time: 0.1038 (0.1169)  time: 1.8730  data: 0.3776  max mem: 12207
Test:  [300/640]  eta: 0:10:44  model_time: 1.0329 (1.0324)  evaluator_time: 0.1113 (0.1180)  time: 1.8477  data: 0.3746  max mem: 12207
Test:  [400/640]  eta: 0:07:33  model_time: 1.0317 (1.0302)  evaluator_time: 0.1142 (0.1204)  time: 1.8868  data: 0.3763  max mem: 12207
Test:  [500/640]  eta: 0:04:23  model_time: 1.0282 (1.0292)  evaluator_time: 0.1034 (0.1195)  time: 1.8577  data: 0.3760  max mem: 12207
Test:  [600/640]  eta: 0:01:15  model_time: 1.0308 (1.0287)  evaluator_time: 0.1124 (0.1189)  time: 1.9121  data: 0.3642  max mem: 1

 17%|█▋        | 1/6 [38:26<3:12:14, 2306.83s/it]

DONE (t=12.62s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.219
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.447
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.184
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.103
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.282
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.383
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.143
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.294
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.313
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.187
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.383
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.470
######

 33%|███▎      | 2/6 [1:17:50<2:34:55, 2324.00s/it]

##########  Evaluation of IDD   ###   IDX   7
Test:  [  0/640]  eta: 0:40:42  model_time: 1.2551 (1.2551)  evaluator_time: 0.1131 (0.1131)  time: 3.8162  data: 1.9802  max mem: 12207
Test:  [100/640]  eta: 0:18:30  model_time: 1.2184 (1.1874)  evaluator_time: 0.1062 (0.1149)  time: 2.0136  data: 0.3706  max mem: 12207
Test:  [200/640]  eta: 0:15:00  model_time: 1.2175 (1.1833)  evaluator_time: 0.0962 (0.1168)  time: 2.0192  data: 0.3918  max mem: 12207
Test:  [300/640]  eta: 0:11:29  model_time: 1.2150 (1.1733)  evaluator_time: 0.1102 (0.1149)  time: 1.9887  data: 0.3504  max mem: 12207
Test:  [400/640]  eta: 0:08:06  model_time: 1.2156 (1.1754)  evaluator_time: 0.1047 (0.1163)  time: 2.0108  data: 0.3783  max mem: 12207
Test:  [500/640]  eta: 0:04:43  model_time: 1.2186 (1.1753)  evaluator_time: 0.0937 (0.1169)  time: 1.9996  data: 0.3619  max mem: 12207
Test:  [600/640]  eta: 0:01:21  model_time: 1.2190 (1.1759)  evaluator_time: 0.1101 (0.1179)  time: 2.0260  data: 0.3774  max mem: 1

 50%|█████     | 3/6 [1:57:21<1:56:53, 2337.90s/it]

##########  Evaluation of IDD   ###   IDX   8
Test:  [  0/640]  eta: 0:51:18  model_time: 1.2551 (1.2551)  evaluator_time: 0.1112 (0.1112)  time: 4.8101  data: 3.0195  max mem: 12207
Test:  [100/640]  eta: 0:18:34  model_time: 1.2188 (1.1857)  evaluator_time: 0.1035 (0.1144)  time: 2.0511  data: 0.3951  max mem: 12207
Test:  [200/640]  eta: 0:15:01  model_time: 1.2189 (1.1889)  evaluator_time: 0.1017 (0.1179)  time: 2.0205  data: 0.3720  max mem: 12207
Test:  [300/640]  eta: 0:11:32  model_time: 1.2201 (1.1834)  evaluator_time: 0.0955 (0.1219)  time: 2.0689  data: 0.3777  max mem: 12207
Test:  [400/640]  eta: 0:08:06  model_time: 1.2207 (1.1819)  evaluator_time: 0.0977 (0.1182)  time: 2.0060  data: 0.3763  max mem: 12207
Test:  [500/640]  eta: 0:04:43  model_time: 1.2190 (1.1819)  evaluator_time: 0.1085 (0.1181)  time: 2.0013  data: 0.3653  max mem: 12207
Test:  [600/640]  eta: 0:01:20  model_time: 1.2155 (1.1803)  evaluator_time: 0.1068 (0.1181)  time: 1.9501  data: 0.3322  max mem: 1

 67%|██████▋   | 4/6 [2:36:51<1:18:15, 2347.56s/it]

##########  Evaluation of IDD   ###   IDX   9
Test:  [  0/640]  eta: 0:42:40  model_time: 1.2395 (1.2395)  evaluator_time: 0.1659 (0.1659)  time: 4.0002  data: 2.0864  max mem: 12207
Test:  [100/640]  eta: 0:18:28  model_time: 1.2194 (1.1809)  evaluator_time: 0.1076 (0.1168)  time: 2.0085  data: 0.3802  max mem: 12207
Test:  [200/640]  eta: 0:15:00  model_time: 1.2200 (1.1817)  evaluator_time: 0.1110 (0.1202)  time: 2.0444  data: 0.3596  max mem: 12207
Test:  [300/640]  eta: 0:11:32  model_time: 1.2207 (1.1813)  evaluator_time: 0.1249 (0.1214)  time: 2.0791  data: 0.3435  max mem: 12207
Test:  [400/640]  eta: 0:08:06  model_time: 1.2169 (1.1818)  evaluator_time: 0.0981 (0.1188)  time: 2.0013  data: 0.3728  max mem: 12207
Test:  [500/640]  eta: 0:04:43  model_time: 1.2198 (1.1819)  evaluator_time: 0.1130 (0.1191)  time: 2.0298  data: 0.3736  max mem: 12207
Test:  [600/640]  eta: 0:01:20  model_time: 1.2211 (1.1809)  evaluator_time: 0.1107 (0.1192)  time: 2.0079  data: 0.3540  max mem: 1

  9%|▉         | 953/10225 [00:41<06:41, 23.07it/s]

KeyboardInterrupt: 

## Finish